# EDA for promotions
Need to understand if promotions are done at an item-family level or store-cluster level, etc.

Also, will determine what to do with NAs

In [ ]:
from config import proj
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Check for missing promo flag. Decide what to do with it.

In [21]:
train = spark.read.parquet(str(proj.Config.paths.get("data_proc").joinpath("train.parquet")))
train.filter("onpromotion is null").agg(sf.max("date")).show()

+----------+
| max(date)|
+----------+
|2014-03-31|
+----------+



Missing promos are only an issue up until March 2014, the feature store currently excludes dates this far back so can ignore them for now.

In [24]:
test = spark.read.parquet(str(proj.Config.paths.get("data_proc").joinpath("test.parquet")))
test.filter("onpromotion is null").agg(sf.max("date")).show()

+---------+
|max(date)|
+---------+
|     null|
+---------+



No missing promo flags in test.

## Discover promotional patterns

In [27]:
pfs = spark.read.parquet(str(proj.Config.paths.get("data_proc").joinpath("feature_store.parquet")))
pfs.show(5)

+----------+----------+----------+--------+---------+----------+-----------+---------+-----+----------+----+-------+--------+------------+---------+---------+---------+----------+
|      date|      city|     state|item_nbr|store_nbr|unit_sales|onpromotion|   family|class|perishable|type|cluster|new_item|cleared_item|event_nat|event_reg|event_loc|event_type|
+----------+----------+----------+--------+---------+----------+-----------+---------+-----+----------+----+-------+--------+------------+---------+---------+---------+----------+
|2016-11-01| El Carmen|    Manabi|  129759|       54|       4.0|      false|GROCERY I| 1032|         0|   C|      3|   false|       false|    false|    false|    false|      null|
|2016-12-26|    Cuenca|     Azuay|  129759|       39|       7.0|      false|GROCERY I| 1032|         0|   B|      6|   false|       false|     true|    false|    false|Additional|
|2016-07-22|     Quito| Pichincha|  129759|        9|       1.0|      false|GROCERY I| 1032|        

In [26]:
pfs.filter("onpromotion is TRUE")

DataFrame[date: date, city: string, state: string, item_nbr: int, store_nbr: int, unit_sales: float, onpromotion: boolean, family: string, class: string, perishable: int, type: string, cluster: string, new_item: boolean, cleared_item: boolean, event_nat: boolean, event_reg: boolean, event_loc: boolean, event_type: string]

In [8]:
pfs.select("date", "onpromotion").groupby("onpromotion").count().show()

+-----------+--------+
|onpromotion|   count|
+-----------+--------+
|       true| 6843833|
|      false|64778430|
+-----------+--------+



In [16]:
pfs.select("onpromotion").distinct().show()

+-----------+
|onpromotion|
+-----------+
|       null|
|       true|
|      false|
+-----------+



In [20]:
pfs.filter("onpromotion is null").agg(sf.max("date")).show()

+----------+
| max(date)|
+----------+
|2014-03-31|
+----------+

